# import library 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
%cd "/content/gdrive/My Drive/flick_dataset_preprocessing"

/content/gdrive/My Drive/flick_dataset_preprocessing


In [3]:
import pandas as pd
import numpy as np
import cv2
import os
import json
import glob
import shutil
import os

# Đọc dữ liệu từ  flickr_logos_27_dataset_training_set_annotation.txt

In [7]:
# đọc dữ liệu từ flie text 
dataset = pd.read_csv("flickr_logos_27_dataset_training_set_annotation.txt",header= None, sep=" ")
# đổi tên các cột sang tên tương ứng
flick = dataset.rename(columns={0:"file_name",1:"categories_names",2: "Training subset",3:"xmin",4:"ymin",5:"xmax",6:"ymax"})
# bỏ cột 7 do là cột trống 
flick = flick.drop([7], axis=1)

In [8]:
flick

,file_name,categories_names,Training subset,xmin,ymin,xmax,ymax
0,144503924.jpg,Adidas,1,38,12,234,142
1,2451569770.jpg,Adidas,1,242,208,413,331
2,390321909.jpg,Adidas,1,13,5,89,60
3,4761260517.jpg,Adidas,1,43,122,358,354
4,4763210295.jpg,Adidas,1,83,63,130,93
...,...,...,...,...,...,...,...
4531,2126991906.jpg,Yahoo,6,15,6,253,54
4532,217288720.jpg,Yahoo,6,136,161,304,222
4533,2472817996.jpg,Yahoo,6,2,4,499,106
4534,2514220918.jpg,Yahoo,6,1,69,342,157


Giải thích ý nghĩa các cột:


*   file_name: tên hình
*   categories_names: tên nhãn của logo
*   Training subset: Training subset 
*   xmin 	ymin 	xmax 	ymax: Coordinates của 2 điểm để tạo bounding box




In [9]:
# sort lại hình ảnh
flick = flick.sort_values(by=['file_name']).reset_index(drop=True)
#thêm vào 2 cột width và height của hình ảnh
flick["width"] = 0
flick["height"] = 0

In [10]:
path = "flickr_logos_27_dataset_images"
# thêm vào width và height cho dataset
for i in range(len(flick)):
  im = cv2.imread(os.path.join(path,flick['file_name'][i])) 
  h, w, c = im.shape
  flick["width"][i] = w
  flick["height"][i] = h

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [11]:
# tạo một cột mới chứa width và height của hình ảnh
flick['image_shape'] = list(zip(flick.width, flick.height))

In [12]:
# dataset sau khi trải qua xử lý
flick

,file_name,categories_names,Training subset,xmin,ymin,xmax,ymax,width,height,image_shape
0,1075391489.jpg,Nbc,6,2,4,77,58,180,66,"(180, 66)"
1,108232382.jpg,Citroen,5,75,29,426,305,500,332,"(500, 332)"
2,108232382.jpg,Citroen,4,75,29,426,305,500,332,"(500, 332)"
3,108232382.jpg,Citroen,3,75,29,426,305,500,332,"(500, 332)"
4,108232382.jpg,Citroen,6,75,29,426,305,500,332,"(500, 332)"
...,...,...,...,...,...,...,...,...,...,...
4531,98498585.jpg,Porsche,6,133,72,382,322,500,375,"(500, 375)"
4532,98498585.jpg,Porsche,5,133,72,382,322,500,375,"(500, 375)"
4533,99816106.jpg,Puma,5,164,35,390,246,500,333,"(500, 333)"
4534,99816106.jpg,Puma,4,164,35,390,246,500,333,"(500, 333)"


In [13]:
#save flick
flick.to_csv(r'/content/gdrive/My Drive/flick_dataset_preprocessing/flick.txt', index=None, sep='\t', mode='a')

# phân tích dataset sau preprocressing

In [15]:
print("số lượng logo: ", len(flick) )
print("số lượng hình: ", flick["file_name"].nunique())
print("sô 1ượng brands: ",flick["categories_names"].nunique())

số lượng logo:  4536
số lượng hình:  809
sô 1ượng brands:  27


In [16]:
print("số lượng hình thuộc từng brand")
counts = flick["file_name"].value_counts()
flick["categories_names"].value_counts()

số lượng hình thuộc tng brand


Pepsi        389
Sprite       281
Cocacola     280
Vodafone     263
Apple        257
Adidas       256
Citroen      210
Unicef       190
Starbucks    188
Heineken     161
Nike         154
RedBull      141
Puma         131
Ford         131
McDonalds    124
Intel        123
Texaco       122
Porsche      120
Fedex        120
BMW          119
DHL          118
Google       117
Yahoo        115
HP           111
Ferrari      105
Nbc          105
Mini         105
Name: categories_names, dtype: int64

In [17]:
flick["image_shape"].value_counts()

(500, 375)    1092
(500, 333)     431
(375, 500)     342
(500, 500)     203
(500, 342)     157
              ... 
(466, 500)       1
(478, 500)       1
(483, 500)       1
(372, 500)       1
(300, 372)       1
Name: image_shape, Length: 313, dtype: int64

# tạo bộ dataset flick dựa trên thông tin từ flick dataframe

In [ ]:
# vì bộ flickr_logos_27_dataset_images chứa những hình không được đánh dấu, ta cần tạo một bộ dataset mới chỉ có hình ảnh nằm trong flick
# đọc tất cả các tên đặt biệt từ flick
# chuyển tất cả các file ảnh từ dataset flickr_logos_27_dataset_images về một folder mới 

src_dir = "/content/gdrive/My Drive/flick_dataset_preprocessing/flickr_logos_27_dataset_images"
dst_dir = "/content/gdrive/My Drive/mmdetection/flick/training/image_2"
uquine_name = flick["file_name"].unique()
for  i in range(len(uquine_name)):
    name = uquine_name[i]
    jpgfile = os.path.join(src_dir,name)
    shutil.copy(jpgfile, dst_dir)

# chuyển dữ liệu từ flickr_logos_27_dataset_training_set_annotation.txt về dạng custom format cho mmdetection 

In [ ]:
#  đọc dữ liệu từ flick và chuyển về dạng format thích hợp cho mmdetection

dst_dir2 = "/content/gdrive/My Drive/mmdetection/flick/training/label_2"
for  i in range(len(uquine_name)):
 
  name = uquine_name[i]
  df3 = flick[flick.file_name == name].reset_index(drop = True)
  text_name = name[:-4]+ ".txt"
  text_name_dir = os.path.join(dst_dir2,text_name)
  with open(text_name_dir, "w") as text_file:
      for  j in range(len(df3)):
        label = "Car"
        x1 = str(df3["xmin"][j])
        y1 = str(df3["ymin"][j])
        x2 = str(df3["xmax"][j])
        y2 = str(df3["ymax"][j])
        strings = label + " " + x1 + " " + y1 + " " + x2 + " " + y2
        text_file.write(strings)
        text_file.write("\n")

# convert sang middle format cho mmdetection

In [18]:
# đọc dữ liệu từ flick dataframe và chuyển về dạng middle format để sử sụng cho mmdetection

df2 = flick
i=0
uquine_name = flick["file_name"].unique()
data_infos = []

for i in range(len(uquine_name)): # loop qua 809 hình trong training
  #tạo list rỗng đễ chứa tọa độ của bounding box và label
  bboxes = []
  labels = []
  
  bboxes_ignore = []
  labels_ignore = []
  #lấy ra tên từng hình
  name = uquine_name[i]
  
  #dùng tên hình tạo dataframe mới chỉ chứa hình đó
  df3 = df2[df2.file_name == name].reset_index(drop = True)

  # lấy ra shape của hình
  w = df3["width"][0]
  h = df3["height"][0]

  #loop qua tất cả các boundind box và label của boundind box trong hình
  #bỏ vào trong 2 list rỗng bboxes và labels
  for j in range(len(df3)):
    x1 = df3["xmin"][j]
    y1 = df3["ymin"][j]
    x2 = df3["xmax"][j]
    y2 = df3["ymax"][j]
    bboxes.append([x1,y1,x2,y2])
    labels.append(1)

  # chuyển về np.array theo yêu cầu của format
  bboxes=np.array(bboxes).astype(np.float32)
  labels=np.array(labels).astype(np.int64)
  
  bboxes_ignore=np.array(bboxes_ignore).astype(np.float32)
  labels_ignore=np.array(labels_ignore).astype(np.int64)
  #tạo một list mới theo format 
  data = [
    {
        'filename':name,
        'width': w,
        'height': h,
        'ann': {
            'bboxes': bboxes,
            'labels': labels,
            'bboxes_ignore': bboxes_ignore,
            'labels_ignore': labels_ignore
        }
    }

  ]

  #thêm lost đó vào list chung
  data_infos.append(data)

# extract to json format

In [19]:
# tạo class MyEncoder để tránh lỗi

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

In [20]:
# extract ra json format và kiểm tra
jstr = json.dumps(data_infos, indent=4,cls=MyEncoder)
print(jstr)

[
    [
        {
            "filename": "1075391489.jpg",
            "width": 180,
            "height": 66,
            "ann": {
                "bboxes": [
                    [
                        2.0,
                        4.0,
                        77.0,
                        58.0
                    ]
                ],
                "labels": [
                    1
                ],
                "bboxes_ignore": [],
                "labels_ignore": []
            }
        }
    ],
    [
        {
            "filename": "108232382.jpg",
            "width": 500,
            "height": 332,
            "ann": {
                "bboxes": [
                    [
                        75.0,
                        29.0,
                        426.0,
                        305.0
                    ],
                    [
                        75.0,
                        29.0,
                        426.0,
                        305.0
                  

In [ ]:
# save về một file mới để sử dụng mmdetection

with open('data_middle_format_2.json', 'w') as outfile:
    json.dump(data_infos, outfile,cls=MyEncoder)